<a href="https://colab.research.google.com/github/demid5111/dialog-intent-dl/blob/master/ipynb/predict_intent_with_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
! tar -xzf "/content/drive/My Drive/Projects/RFFI2016/dataset.tar.gz" -C /content
! ls "/content/feature_and_vector_seq" | wc -l

13156


In [0]:
! git clone https://github.com/demid5111/dialog-intent-dl.git
! mv dialog-intent-dl dialog_intent_dl
! pip install -r dialog_intent_dl/requirements.txt
! pip install tensorflow-gpu==1.13.1
! pip install numpy==1.15.4
# restart runtime
import os
os.kill(os.getpid(), 9)

Cloning into 'dialog-intent-dl'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 185 (delta 19), reused 20 (delta 9), pack-reused 148
Receiving objects: 100% (185/185), 178.81 KiB | 13.75 MiB/s, done.
Resolving deltas: 100% (91/91), done.
     |████████████████████████████████| 1.4MB 5.1MB/s 
     |████████████████████████████████| 4.5MB 38.4MB/s 
     |████████████████████████████████| 22.6MB 55.9MB/s 
     |████████████████████████████████| 1.6MB 43.0MB/s 
     |████████████████████████████████| 13.9MB 36.9MB/s 
     |████████████████████████████████| 11.8MB 19.9MB/s 
     |████████████████████████████████| 31.2MB 25.5MB/s 
     |████████████████████████████████| 143kB 42.2MB/s 
     |████████████████████████████████| 345.2MB 49kB/s 
     |████████████████████████████████| 3.2MB 31.0MB/s 
     |████████████████████████████████| 368kB 44.8MB/s 
  Created wheel for nltk: filename=nltk-

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
! cd dialog_intent_dl && git pull
# import pandas as pd
# df1 = pd.read_hdf("/content/feature_and_vector_seq/8-comm_rosbalt_39_79701_output_10.h5", engine="python", encoding='cp1251', mode='a')

Mounted at /content/drive
Already up to date.


In [0]:
import os, random, json, re
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
def load_Doc2Vec_value(str1):
  pattern1 = re.compile('\n')
  pattern2 = re.compile(' ')
  str2 = pattern1.sub('', str1)
  str3 = pattern2.sub(', ', str2)
  return json.loads(str3)

class SequenceGenerator():
    def __init__(self, data_path, intent_index, max_sequence_length, validation_split, only_last=False,
                 random_state=None):
        self.data_path = data_path
        self.max_sequence_length = max_sequence_length
        self.intent_index = intent_index
        self.num_intents = max(intent_index.values()) + 1
        self.only_last = only_last
        self.file_list_train, self.file_list_test = self._split(validation_split, random_state=random_state)

    def _file2sequence(self, file_and_path):
        intent_list = []
        embed_list = []
        df1 = pd.read_hdf(file_and_path, engine="python", encoding='cp1251')

        for key, row in df1.iterrows():
            intent = row['Intent analysis']
            doc2vec = load_Doc2Vec_value(row["Doc2Vec value"])
            if intent:
                intent_char = intent[0].lower()
            else:
                intent_char = ""
            intent_list.append(self.intent_index[intent_char])
            embed_list.append(doc2vec)
        return intent_list, embed_list

    def _split(self, validation_split, random_state=None):
        file_list = os.listdir(self.data_path)
        file_list_train, file_list_test = train_test_split(file_list, test_size=validation_split,
                                                           random_state=random_state)
        return file_list_train, file_list_test

    def __build_intent_sequence(self, dialogs_list):
        sequence_list = []
        for dialog in dialogs_list:
            sequence = []
            for intent in dialog['Intent analysis'].values:
                sequence.append(self.intent_index[intent])
            sequence_list.append(sequence)
        paded_sequences = pad_sequences(sequence_list, maxlen=self.max_sequence_length)
        return paded_sequences

    def generate_batch(self, batch_size, subset='training'):
        if subset == 'training':
            file_list = self.file_list_train
        elif subset == 'validation':
            file_list = self.file_list_test
        f_i = 0

        while True:
            i = 0
            sequence_batch, embed_batch = [], []
            while i < batch_size:
                if f_i == len(file_list):
                    f_i = 0
                    random.shuffle(file_list)
                file_and_path = os.path.join(self.data_path, file_list[f_i])
                intent_sequence, embed_list = self._file2sequence(file_and_path)
                if len(intent_sequence) > self.max_sequence_length:
                    for ii in range(len(intent_sequence) - self.max_sequence_length + 1):
                        sequence_i = intent_sequence[ii:self.max_sequence_length + ii]
                        sequence_batch.append(sequence_i)
                        embed_list_i = embed_list[ii:self.max_sequence_length + ii]
                        embed_batch.append(embed_list_i)
                        i += 1
                else:
                    sequence_batch.append(intent_sequence)
                    embed_batch.append(embed_list)
                    # print(len(embed_list),len(embed_list[0]))
                    i += 1
                f_i += 1
                # print("embed_list",len(embed_list))
                
            paded_sequences = pad_sequences(sequence_batch, maxlen=self.max_sequence_length, dtype='float')
            paded_embeds = pad_sequences(embed_batch, maxlen=self.max_sequence_length, dtype='float')
            # print(paded_embeds)
            inputs = paded_sequences[:, :-1]
            if self.only_last:
                labels = to_categorical(paded_sequences[:, -1:],
                                        num_classes=self.num_intents)  # for build_CNN_model build_MLP_model
            else:
                labels = to_categorical(paded_sequences[:, 1:],
                                        num_classes=self.num_intents)  # for build_BiRNN_model build_RNN_model
            concat_inputs = np.concatenate((np.expand_dims(inputs,-1), np.array(paded_embeds[:, :-1])), 2)
            yield concat_inputs, labels



intents = {"": 0, " ": 0, "а": 1, "a": 1, "б": 2, "в": 3, "г": 4, "д": 5,
               "е": 6, "e": 6, "ж": 7, "з": 8, "3": 8, "и": 9, "к": 10,
               "л": 11, "м": 12, "н": 13, "о": 14, "п": 15,
               "р": 16, "с": 17, "т": 18, "у": 19, "ф": 20,
               "х": 21, "ц": 22, "ч": 23, "ш": 24, "щ": 25}
batch_size = 7
max_sequence_length = 5
validation_split = 0.2
random_state = 42
sg = SequenceGenerator("/content/feature_and_vector_seq", intents, max_sequence_length, validation_split,
                                    only_last=False,
                                    random_state=random_state)
inputs, labels = next(sg.generate_batch(batch_size, subset='training'))
inputs.shape, labels.shape

((7, 4, 301), (7, 4, 26))

In [0]:
import os
import tensorflow as tf
import keras
from keras.layers import Dense, Input, GlobalMaxPooling1D, Flatten, TimeDistributed, Bidirectional
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, Dropout, Activation, Permute, Lambda
from keras import regularizers
from keras.models import Model, Sequential
from keras.callbacks import TensorBoard
from keras.backend import permute_dimensions, squeeze, expand_dims
# from dialog_intent_dl.ipynb.keras_handler.sequence_generator import SequenceGenerator
# from dialog_intent_dl.ipynb.keras_handler.sequence_generator import PredictIntent
print(tf.__version__)

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph
    
def save_model(session, model, name):
    frozen_graph = freeze_session(session, output_names=[out.op.name for out in model.outputs])
    tf.train.write_graph(frozen_graph, "../models", "{}".format(name), as_text=False)
    
class PredictIntent():
    intents = {"": 0, " ": 0, "а": 1, "a": 1, "б": 2, "в": 3, "г": 4, "д": 5,
               "е": 6, "e": 6, "ж": 7, "з": 8, "3": 8, "и": 9, "к": 10,
               "л": 11, "м": 12, "н": 13, "о": 14, "п": 15,
               "р": 16, "с": 17, "т": 18, "у": 19, "ф": 20,
               "х": 21, "ц": 22, "ч": 23, "ш": 24, "щ": 25}
    general_intents = {"": 0, " ": 0, "а": 1, "a": 1, "б": 1, "в": 1, "г": 1, "д": 1,  # Информативно-воспроизводящий
                       "е": 2, "e": 2, "ж": 2, "з": 2, "3": 2, "и": 2, "к": 2,  # Эмотивно-консолидирующий
                       "л": 3, "м": 3, "н": 3, "о": 3, "п": 3,  # Манипулятивный тип, доминирование
                       "р": 4, "с": 4, "т": 4, "у": 4, "ф": 4,  # Волюнтивно-директивный
                       "х": 5, "ц": 5, "ч": 5, "ш": 5, "щ": 5}  # Контрольно-реактивный
    batch_size = 13
    max_sequence_length = 5
    intent_embedding_dim = 10
    input_embedding_dim =300
    num_units = 30
    validation_split = 0.2
    random_state = 42
    tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False,
                     write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

    def __init__(self, is_general=False, data_path = "../feature_and_vector_seq"):
        self.data_path = data_path
        if is_general:
            self.intent_index = self.general_intents
        else:
            self.intent_index = self.intents
        self.num_intents = max(self.intent_index.values()) + 1
        self.sg = SequenceGenerator(self.data_path, self.intent_index, self.max_sequence_length, self.validation_split,
                                    only_last=False,
                                    random_state=self.random_state)
        print('num_intents', self.num_intents)

    def build_CNN_model(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)

        # sequence_input = Input(shape=(self.max_sequence_length - 1,), dtype='int32')
        concat_input = Input(shape=(self.max_sequence_length - 1,self.input_embedding_dim+1), dtype='float')
        def lambda_fun(x) : 
          split1, split2 = tf.split(x, [1, 300], 2)
          return split1, split2
        sequence_input, embedded_input = Lambda(lambda_fun)(concat_input)
        # sequence_input = concat_input[:, :, 0]
        # embedded_input = concat_input[:, :, 1:]
        embedded_sequences = embedding_layer(sequence_input)
        #         x_p = permute_dimensions(embedded_sequences, pattern=[0, 2, 1])

        x = Conv1D(128, 2, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(embedded_sequences)
        x = MaxPooling1D(1)(x)
        x = Dropout(0.2)(x)
        x = Conv1D(128, 3, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = GlobalMaxPooling1D()(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        preds = Dense(self.num_intents, activation='softmax')(x)

        model = Model(concat_input, preds)
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        self.sg.only_last = True
        return self.model

    def build_MLP_model(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)
        embedded_input = Input(shape=(self.max_sequence_length - 1, self.input_embedding_dim), dtype='float')
        model = Sequential()
        model.add(embedding_layer)
        model.add(Flatten())
        model.add(Dense(128, activation='relu', name="Dense1"
                        #                         , activity_regularizer=regularizers.l1(0.009)
                        , kernel_regularizer=regularizers.l2(0.0001)
                        #                         , bias_regularizer = regularizers.l2(0.009)
                        ))  #
        #         model.add(Dropout(0.2))
        model.add(Dense(self.num_intents, activation='softmax', name="Dense2"
                        #                         , activity_regularizer=regularizers.l1(0.009)
                        , kernel_regularizer=regularizers.l2(0.0001)
                        #                         , bias_regularizer = regularizers.l2(0.009)
                        ))
        #         model.add(Dropout(0.2))
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        self.sg.only_last = True
        return self.model

    def build_RNN_model(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)

        sequence_input = Input(shape=(self.max_sequence_length - 1,), dtype='int32')
        embedded_input = Input(shape=(self.max_sequence_length - 1, self.input_embedding_dim), dtype='float')
        embedded_sequences = embedding_layer(sequence_input)

        #         x_t = permute_dimensions(embedded_sequences, pattern=[0, 2, 1])
        #         print('embedded_sequences',embedded_sequences.shape)
        #         print('x_t',x_t.shape)

        x = LSTM(self.num_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embedded_sequences)
        x = LSTM(self.num_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(x)
        preds = TimeDistributed(Dense(self.num_intents, activation='softmax'))(x)
        #         preds = Dense(self.num_intents, activation='softmax')(x)
        #         print('preds.shape',preds.shape)

        model = Model(sequence_input, embedded_input, preds)
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        self.sg.only_last = False
        return self.model

    def build_BiRNN_model(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)
        embedded_input = Input(shape=(self.max_sequence_length - 1, self.input_embedding_dim), dtype='float')
        model = Sequential()
        model.add(embedding_layer)
        #         model.add(Permute([1, 2]))
        model.add(Bidirectional(LSTM(self.num_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
        model.add(Bidirectional(LSTM(self.num_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
        #         model.add(Bidirectional(LSTM(self.num_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
        model.add(
            TimeDistributed(Dense(self.num_intents, activation='softmax', name="Dense2"), name="TimeDistributed2"))
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        return self.model

    def RNN_attention(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)
        embedded_input = Input(shape=(self.max_sequence_length - 1, self.input_embedding_dim), dtype='float')
        model1 = Sequential()
        model1.add(embedding_layer)
        model1.add(LSTM(self.num_units, return_sequences=True))

        model2 = Sequential()
        model2.add(Dense(input_dim=input_dim, output_dim=step))
        model2.add(Activation('softmax'))  # Learn a probability distribution over each  step.
        # Reshape to match LSTM's output shape, so that we can do element-wise multiplication.
        model2.add(RepeatVector(self.num_units))
        model2.add(Permute(2, 1))

        model = Sequential()
        model.add(
            Merge([model1, model2], 'mul'))  # Multiply each element with corresponding weight a[i][j][k] * b[i][j]
        model.add(TimeDistributedMergeinput_dim('sum'))  # Sum the weighted elements.

        attention = Dense(1, activation='tanh')(activations)
        attention = Flatten()(attention)
        attention = Activation('softmax')(attention)
        attention = RepeatVector(units)(attention)
        attention = Permute([2, 1])(attention)

        sent_representation = merge([activations, attention], mode='mul')
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        self.sg = SequenceGenerator(self.data_path, self.intent_index, self.max_sequence_length, self.validation_split,
                                    only_last=False)
        return self.model

    def fit_generator(self, epochs):
        history_nn = self.model.fit_generator(
            generator=self.sg.generate_batch(self.batch_size, subset='training'),
            steps_per_epoch=len(os.listdir(self.data_path)) * (1 - self.validation_split) // self.batch_size,
            epochs=epochs,
            validation_data=self.sg.generate_batch(self.batch_size, subset='validation'),
            validation_steps=len(os.listdir(self.data_path)) * self.validation_split // self.batch_size + 1,
            callbacks=[self.tb]
        )
        return history_nn

1.13.1


In [0]:
PATH_TO_DATASET = "/content/feature_and_vector_seq"
pi = PredictIntent(data_path = PATH_TO_DATASET, is_general = False)
pi.batch_size = 7
pi.max_sequence_length = 5
pi.intent_embedding_dim = 10
pi.num_units = 30
pi.validation_split = 0.2
pi.random_state = 42

num_intents 26


In [0]:
pi.build_CNN_model()
history_CNN = pi.fit_generator(epochs = 1)
from keras import backend as K
save_model(K.get_session(), pi.model, 'history_CNN.pb')

AttributeError: ignored

In [0]:
pi.RNN_attention()
history_BiRNN = pi.fit_generator(epochs = 1)
from keras import backend as K
save_model(K.get_session(), pi.model, 'RNN_attention.pb')

NameError: ignored

In [0]:
§import pandas as pd
import re, json

def load_Doc2Vec_value(str1):
  pattern1 = re.compile('\n')
  pattern2 = re.compile(' ')
  str2 = pattern1.sub('', str1)
  str3 = pattern2.sub(', ', str2)
  return json.loads(str3)

file_and_path = "/content/feature_and_vector_seq/1-comm_inosmi_111_192344_output_10.h5"
df1 = pd.read_hdf(file_and_path, engine="python", encoding='cp1251')
df1["Doc2Vec"] = df1["Doc2Vec value"].apply(load_Doc2Vec_value)
df1


,ID of comment,ID of post,Likes,Intent analysis,Content analysis,Distance to parent,Distance to post,Doc2Vec value,Doc2Vec
0,192344,192344,0,,,0,1.1102230246251565e-16,[0.001903 0.008161 0.003022 0.021471 0.001911 ...,"[0.001903, 0.008161, 0.003022, 0.021471, 0.001..."
1,192366,192344,0,м,1,0.9047975317843413,0.9047975317843413,[0.015871 0.063059 0.027442 0.005918 0.067336 ...,"[0.015871, 0.063059, 0.027442, 0.005918, 0.067..."
2,192369,192344,1,а,3,0.9263160364648992,0.949031225401245,[-0.050479 -0.020054 0.010354 0.009482 -0.0072...,"[-0.050479, -0.020054, 0.010354, 0.009482, -0...."
3,192370,192344,0,ч,3,0.8536771566133271,1.0161022628113872,[-0.003400 0.004530 -0.011841 0.009115 0.01707...,"[-0.0034, 0.00453, -0.011841, 0.009115, 0.0170..."
4,192371,192344,1,н,4,0.4652375928796576,1.1818890054062847,[-0.017641 0.020937 -0.043087 0.017828 0.03717...,"[-0.017641, 0.020937, -0.043087, 0.017828, 0.0..."
5,192372,192344,0,р,4,0.5322816918635409,0.918104637002647,[0.014156 0.003831 0.001797 0.009888 0.004583 ...,"[0.014156, 0.003831, 0.001797, 0.009888, 0.004..."
6,192377,192344,0,ф,3,0.9230922955341927,0.5331857866510279,[-0.026679 -0.030724 0.026273 0.014249 0.00759...,"[-0.026679, -0.030724, 0.026273, 0.014249, 0.0..."
7,192381,192344,0,ч,4,0.8990398151377282,0.9101620933407938,[0.008495 0.029132 -0.002679 -0.007744 0.01133...,"[0.008495, 0.029132, -0.002679, -0.007744, 0.0..."
8,192395,192344,1,ч,3,1.0171192821636792,0.7188240607558654,[-0.009065 -0.010122 0.035569 -0.002556 -0.003...,"[-0.009065, -0.010122, 0.035569, -0.002556, -0..."


In [0]:
import numpy as np
import pandas as pd
data = np.random.rand(200,2)
expected = np.random.randint(2, size=200).reshape(-1,1)

dataFrame = pd.DataFrame(data, columns = ['a','b'])
expectedFrame = pd.DataFrame(expected, columns = ['expected'])

dataFrameTrain, dataFrameTest = dataFrame[:100],dataFrame[-100:]
expectedFrameTrain, expectedFrameTest = expectedFrame[:100],expectedFrame[-100:]

def generator(X_data, y_data, batch_size):

  samples_per_epoch = X_data.shape[0]
  number_of_batches = samples_per_epoch/batch_size
  counter=0
  while 1:
    X_batch = np.array(X_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
    y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
    counter += 1
    yield (X_batch,X_batch),y_batch

    #restart counter to yeild data in the next epoch as well
    if counter >= number_of_batches:
        counter = 0

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution1D, Convolution2D, MaxPooling2D
from keras.utils import np_utils

inputs1 = Input(shape=(dataFrame.shape[1],))
# inputs2 = Input(shape=(dataFrame.shape[1],))
x = Dense(12, activation='relu', input_dim=dataFrame.shape[1])(inputs1)
preds = Dense(1, activation='sigmoid')(x)
model = Model(inputs1, preds)
# model = Sequential()
# model.add(Dense(12, activation='relu', input_dim=dataFrame.shape[1]))
# model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

#Train the model using generator vs using the full batch
batch_size = 8

model.fit_generator(generator(dataFrameTrain,expectedFrameTrain,batch_size), epochs=3,steps_per_epoch = dataFrame.shape[0]/batch_size, validation_data=generator(dataFrameTest,expectedFrameTest,batch_size*2),validation_steps=dataFrame.shape[0]/batch_size*2)


Epoch 1/3


AttributeError: ignored